# Hybrid Retrieval System for RAG - Optimized Setup

Complete hybrid retrieval system combining:
- **Dense Retrieval**: FAISS + BGE-M3 embeddings
- **Sparse Retrieval**: BM25 keyword-based search  
- **Hybrid Mode**: Configurable weighted combination (default 70% dense, 30% sparse)
- **Evaluation Metrics**: 6 standard IR metrics for performance assessment

Ready to integrate with your generation model for complete RAG pipeline!

In [1]:
!pip install torch transformers sentence-transformers rank_bm25 nltk faiss-cpu FlagEmbedding tqdm numpy matplotlib seaborn -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.9/163.9 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.0/149.0 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.6 MB/s eta 0:00:00


## Environment Setup

All dependencies installed and libraries imported

In [ ]:
import faiss
import nltk
from app.utils.retrievers.retriever import Retriever

faiss.omp_set_num_threads(12)
nltk.download('punkt_tab', quiet=True)

True

## RetrievalEvaluator Class

Unified retriever with three methods:
- `dense(query, k)` - FAISS + BGE-M3 only
- `sparse(query, k)` - BM25 keyword search only
- `hybrid(query, k, dense_weight, sparse_weight)` - Configurable combination
- `get_article_metadata(doc_id, path)` - Extract article metadata from JSON

## RetrievalEvaluator Class

Unified retriever with three methods:
- `dense(query, k)` - FAISS + BGE-M3 only
- `sparse(query, k)` - BM25 keyword search only
- `hybrid(query, k, dense_weight, sparse_weight)` - Configurable combination
- `get_article_metadata(doc_id, path)` - Extract article metadata from JSON

## Initialize Retriever

Load BGE-M3 model and setup retriever with all three modes ready

In [3]:
from FlagEmbedding import BGEM3FlagModel

print("Loading BGE-M3 model...")
retriever_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
print("✓ Model loaded successfully!")


metric_type = 'ip'
index_path = 'data/m3_legal_faiss.index'
qa_dataset_path = 'data/evaluation_data/law_qa_dataset_validated.json'
documents_path = 'data/saudi_laws_scraped.json'


print(f"\nInitializing retriever with {metric_type.upper()} metric...")
retriever = Retriever(
    faiss_index_path=index_path,
    documents_path=documents_path,
    embeddings_model=retriever_model,
    metric_type=metric_type
)
print("✓ Retriever ready with dense, sparse, and hybrid modes!")

Loading BGE-M3 model...


Fetching 30 files: 100%|██████████| 30/30 [00:00<00:00, 63872.65it/s]


✓ Model loaded successfully!

Initializing retriever with IP metric...
Initializing Retriever...
Loading documents and metadata...
✓ Loaded 16371 documents with metadata
✓ BM25 initialized successfully!
✓ Retriever ready with dense, sparse, and hybrid modes!


In [4]:
test_query = "ما هي أحكام العقود في النظام السعودي؟"

def display_results(results, query_scores, mode_name):
    """Display retrieval results with metadata."""
    print(f"\n{'='*80}")
    print(f"🔹 {mode_name}")
    print(f"{'='*80}")

    for rank, (idx, score) in enumerate(zip(results, query_scores), 1):
        meta = retriever.get_article_metadata(idx)
        if meta:
            print(f"\n{rank}. Doc ID: {idx} | Score: {score:.4f}")
            print(f"   📂 {meta['category']} > {meta['subcategory']}")
            print(f"   📋 {meta['system']}")
            print(f"   📋 Brief: {meta['system_brief'][:80]}...")
            print(f"   📑 Part: {meta['part']}")
            print(f"   📝 {meta['title']}")
            print(f"   📄 {meta['text'][:100]}...")


print(f"Query: {test_query}\n")

dense_scores, dense_indices = retriever.dense(test_query, k=5)
display_results(dense_indices, dense_scores, "DENSE RETRIEVAL (FAISS + BGE-M3)")

sparse_scores, sparse_indices = retriever.sparse(test_query, k=5)
display_results(sparse_indices, sparse_scores, "SPARSE RETRIEVAL (BM25)")

hybrid_scores, hybrid_indices = retriever.hybrid(test_query, k=5, dense_weight=0.7, sparse_weight=0.3)
display_results(hybrid_indices, hybrid_scores, "HYBRID RETRIEVAL (70% Dense + 30% Sparse)")


print(f"\n{'='*80}")
print("📊 RETRIEVAL SUMMARY")
print(f"{'='*80}")
print(f"Total documents: {len(retriever.doc_metadata)}")
print(f"Dense scores: [{dense_scores.min():.4f}, {dense_scores.max():.4f}]")
print(f"Sparse scores: [{sparse_scores.min():.4f}, {sparse_scores.max():.4f}]")
print(f"Hybrid scores: [{hybrid_scores.min():.4f}, {hybrid_scores.max():.4f}]")

dense_set = set(dense_indices)
sparse_set = set(sparse_indices)
hybrid_set = set(hybrid_indices)

print(f"\n🔄 RETRIEVAL OVERLAP")
print(f"Dense ∩ Sparse: {len(dense_set & sparse_set)}/5")
print(f"Dense ∩ Hybrid: {len(dense_set & hybrid_set)}/5")
print(f"Sparse ∩ Hybrid: {len(sparse_set & hybrid_set)}/5")
print(f"All three: {len(dense_set & sparse_set & hybrid_set)}/5")
print(f"{'='*80}")

Query: ما هي أحكام العقود في النظام السعودي؟



You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



🔹 DENSE RETRIEVAL (FAISS + BGE-M3)

1. Doc ID: 11534 | Score: 1.0000
   📂 أنظمة عادية > المال والرقابة
   📋 نظام المنافسات والمشتريات الحكومية
   📋 Brief: يتضمن النظام الآتي :
المبادئ الأساسية وأهداف النظام - تقديم العروض وفتح المظاريف...
   📑 Part: main
   📝 المادة السابعة والثلاثون
   📄 المادة السابعة والثلاثون
تدفع قيمة العقود بالريال السعودي . ويجوز أن تدفع بأي عملة أخرى بعد التنسيق ...

2. Doc ID: 3514 | Score: 0.6511
   📂 أنظمة عادية > التجارة والاقتصاد والاستثمار
   📋 نظام  المنافسات و المشتريات الحكومية
   📋 Brief: نظام المنافسات والمشتريات الحكومية 1440هـ....
   📑 Part: الفصل الثالث المقابل المالي
   📝 المادة الرابعة والستون
   📄 المادة الرابعة والستون
تدفع قيمة العقود بالريال السعودي، ويجوز أن تحدد الجهة الحكومية في وثائق المنا...

3. Doc ID: 11812 | Score: 0.5298
   📂 أنظمة عادية > المال والرقابة
   📋 نظام البنك المركزي السعودي
   📋 Brief: حل هذا النظام محل نظام مؤسسة النقد العربي السعودي لعام 1377هـ ،
وتضمن : التعريفا...
   📑 Part: الفصل السادس:  أحكام ختامية
   📝 المادة ا

In [11]:
re_ranked_scores, re_ranked_indices = retriever.re_ranked_search(test_query,relevant_terms=['العقد'], subcategory_filters=['التجارة والاقتصاد والاستثمار'], k=5, keyword_boost=0.5 )
display_results(re_ranked_indices, re_ranked_indices, "re_ranked RETRIEVAL")

Filtering to 506 documents in subcategories: ['التجارة والاقتصاد والاستثمار']

🔹 re_ranked RETRIEVAL

1. Doc ID: 14598 | Score: 14598.0000
   📂 تنظيمات، وترتيبات تنظيمية > التجارة والاقتصاد والاستثمار
   📋 تنظيم المركز السعودي للتحكيم التجاري
   📋 Brief: تنظيم المركز السعودي للتحكيم التجاري...
   📑 Part: الفصل الأول: طبيعة المركز ومقره واختصاصه
   📝 المادة الأولى
   📄 المادة الأولى
يقصد بالكلمات والعبارات الآتية -أينما وردت في هذا التنظيم- المعاني الموضحة أمام كل منه...

2. Doc ID: 14783 | Score: 14783.0000
   📂 تنظيمات، وترتيبات تنظيمية > التجارة والاقتصاد والاستثمار
   📋 تنظيم هيئة التأمين
   📋 Brief: تضمن التنظيم:
تعريفات، الشخصية الاعتبارية للهيئة، أهداف الهيئة ومهامها، مجلس إدا...
   📑 Part: main
   📝 المادة التاسعة
   📄 المادة التاسعة
1.
تتكون موارد الهيئة من الآتي:
أ- المقابل المالي الذي تتقاضاه نظير الخدمات والأعمال ...

3. Doc ID: 14611 | Score: 14611.0000
   📂 تنظيمات، وترتيبات تنظيمية > التجارة والاقتصاد والاستثمار
   📋 تنظيم المركز السعودي للتحكيم التجاري
   📋 Brief: تنظيم 

## RAG System Integration

Complete LangChain-based RAG system for legal question answering

In [ ]:
from langchain_openai import ChatOpenAI
from rag import LegalAssistantRAG, RAGConfig
import os

# Set your Gemini API key
# You can either set it as an environment variable or directly here
# os.environ["GOOGLE_API_KEY"] = "your-api-key-here"

rag_config = RAGConfig(k=12)

# Initialize Gemini model
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",  # or "gemini-2.5-pro" for more capable model
    temperature=0.1,
    google_api_key=os.getenv("GOOGLE_API_KEY")  # or pass your API key directly
)

# Initialize the RAG system
rag_system = LegalAssistantRAG(
    retriever=retriever,
    llm=llm,
    config=rag_config
)

print("✓ RAG system initialized with Gemini!")

2025-10-23 01:40:55,621 - rag - INFO - ✓ RAG chain initialized
2025-10-23 01:40:55,621 - rag - INFO - ✓ Legal Assistant RAG initialized


In [8]:
rag_system.answer("ما هو مصدر الإفتاء في المملكة العربية السعودية؟")


2025-10-23 01:40:57,371 - rag - INFO - Answering question: 'ما هو مصدر الإفتاء في المملكة العربية السعودية؟' with 0 history messages
2025-10-23 01:41:02,433 - rag - INFO - Detected 1 tool call(s).
2025-10-23 01:41:02,434 - rag - INFO - Executing tool: legal_search with args: {'query': 'مصدر الإفتاء في المملكة العربية السعودية'}
2025-10-23 01:41:02,435 - rag - INFO - Performing search for: 'مصدر الإفتاء في المملكة العربية السعودية'
2025-10-23 01:41:02,502 - rag - INFO - Found 12 articles for 'مصدر الإفتاء في المملكة العربية السعودية'
2025-10-23 01:41:02,503 - rag - INFO - Sending tool results back to LLM for final synthesis.


'مصدر الإفتاء في المملكة العربية السعودية هو:\n\n- **كتاب الله تعالى**  \n- **سنة رسوله صلى الله عليه وسلم**\n\nوقد نصت المادة الخامسة والأربعون من **النظام الأساسي للحكم** على ذلك، مشيرة إلى أن هذين المصدرَين هما الأساس في الإفتاء، كما يُحدد النظام ترتيب هيئة كبار العلماء وإدارة البحوث العلمية والإفتاء واختصاصاتها.'

In [9]:
print(rag_system.answer("ما هو مصدر الإفتاء في المملكة العربية السعودية؟"))

2025-10-23 01:41:04,942 - rag - INFO - Answering question: 'ما هو مصدر الإفتاء في المملكة العربية السعودية؟' with 4 history messages
2025-10-23 01:41:07,771 - rag - INFO - No tool call required. Returning direct answer.


مصدر الإفتاء في المملكة العربية السعودية هو:

- **كتاب الله تعالى**  
- **سنة رسوله صلى الله عليه وسلم**

وقد نصت المادة الخامسة والأربعون من **النظام الأساسي للحكم** على ذلك، مشيرة إلى أن هذين المصدرَين هما الأساس في الإفتاء، كما يُحدد النظام ترتيب هيئة كبار العلماء وإدارة البحوث العلمية والإفتاء واختصاصاتها.
